Login stuff - once this cell runs you should be able to leave the rest running

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
pth = '/content/gdrive/MyDrive/'

! pip install -q wandb

Mounted at /content/gdrive/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [2]:
import wandb

wandb.login(relogin=True)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
! git clone --branch liam https://github.com/liamchalcroft/ano-synth/

Cloning into 'ano-synth'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (359/359), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 1102 (delta 229), reused 273 (delta 183), pack-reused 743
Receiving objects: 100% (1102/1102), 213.76 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (696/696), done.


In [4]:
# ! pip install -q --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
! pip install -q --upgrade pytorch-ignite
! pip install -q monai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.4/272.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.1 MB/s eta 0:00:00


In [5]:
import multiprocessing
cores = multiprocessing.cpu_count()
print('Using CPU with total of {} cores.'.format(cores))

Using CPU with total of 8 cores.


# Training

In [6]:
! tar -xzf /content/gdrive/MyDrive/ano-synth-experiments/data/oasis2d_trainval.tar.gz --skip-old-files

In [7]:
#model_name = 'gaussvae-flat'
#model = 'GaussVAE'
# model_name = 'liam-samba'
# model = 'SAMBA'
model_name = 'AE-flat-mix'
model = 'AE'
rootdir = '/content/gdrive/MyDrive/ano-synth-experiments/'


In [8]:
! ls {rootdir+model_name}

ls: cannot access '/content/gdrive/MyDrive/ano-synth-experiments/AE-flat-mix': No such file or directory


In [9]:
! cd ano-synth; git pull

Already up to date.


In [10]:
! python ano-synth/train.py --resume --mix --name {model_name} --model {model} --root {rootdir} --workers {cores} --epochs 2000 --val_interval 50 --batch_size 8 --flat_latent

2024-02-01 11:22:17.967345: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 11:22:17.967394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 11:22:17.968958: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 11:22:19.497663: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Train Images: 167990
Val Images: 33600

Using device: cuda
Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
ckpts: ['/content/gdrive/MyDrive/ano-synth-experiments/mix-AE-flat-mix/checkpoint.pt']

Resuming from epoch #1972 with Wan

# Prediction / Val

In [11]:
import os, glob
import torch
import tqdm
import monai as mn
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt

In [12]:
# model_name = 'gaussvae-flat'
# model = 'GaussVAE'
model_name = 'AE-flat-mix'
model = 'AE'
rootdir = '/content/gdrive/MyDrive/ano-synth-experiments/'

In [13]:
! cd ano-synth; git pull

Already up to date.


## Healthy - IXI

### T1 - In-domain

In [ ]:
! wget http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T1.tar

--2023-12-26 23:03:56--  http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T1.tar
Resolving biomedic.doc.ic.ac.uk (biomedic.doc.ic.ac.uk)... 146.169.22.31
Connecting to biomedic.doc.ic.ac.uk (biomedic.doc.ic.ac.uk)|146.169.22.31|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T1.tar [following]
--2023-12-26 23:03:57--  https://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T1.tar
Connecting to biomedic.doc.ic.ac.uk (biomedic.doc.ic.ac.uk)|146.169.22.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4840816640 (4.5G) [application/x-tar]
Saving to: ‘IXI-T1.tar’

IXI-T1.tar          100%[===================>]   4.51G  13.7MB/s    in 5m 53s  

2023-12-26 23:09:51 (13.1 MB/s) - ‘IXI-T1.tar’ saved [4840816640/4840816640]



In [ ]:
! mkdir IXI-T1; tar -xf IXI-T1.tar -C IXI-T1

In [ ]:
! cd ano-synth; git pull

Already up to date.


In [ ]:
! python ano-synth/predict.py --flat_latent --name {model_name} --model {model} --root {rootdir} --workers {cores} --amp --data ./IXI-T1 --data_name IXI-T1

2023-12-26 23:16:13.320820: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-26 23:16:13.320872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-26 23:16:13.322054: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-26 23:16:14.347561: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Using device: cuda
Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB

Total Images: 581
100% 581/581 [22:03<00:00,  2.28s/it]


### T2 - Out-of-domain

In [ ]:
! wget http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T2.tar

URL transformed to HTTPS due to an HSTS policy
--2023-12-26 23:10:17--  https://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T2.tar
Resolving biomedic.doc.ic.ac.uk (biomedic.doc.ic.ac.uk)... 146.169.22.31
Connecting to biomedic.doc.ic.ac.uk (biomedic.doc.ic.ac.uk)|146.169.22.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3853445120 (3.6G) [application/x-tar]
Saving to: ‘IXI-T2.tar’

IXI-T2.tar          100%[===================>]   3.59G  11.6MB/s    in 4m 42s  

2023-12-26 23:15:00 (13.0 MB/s) - ‘IXI-T2.tar’ saved [3853445120/3853445120]



In [ ]:
! mkdir IXI-T2; tar -xf IXI-T2.tar -C IXI-T2

In [ ]:
! python ano-synth/predict.py --flat_latent --name {model_name} --model {model} --root {rootdir} --workers {cores} --amp --data ./IXI-T2 --data_name IXI-T2

2023-12-26 23:38:29.070727: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-26 23:38:29.070779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-26 23:38:29.071962: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-26 23:38:30.085499: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Using device: cuda
Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB

Total Images: 578
100% 578/578 [18:47<00:00,  1.95s/it]


### PD - Out-of-domain

In [ ]:
! wget http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-PD.tar

URL transformed to HTTPS due to an HSTS policy
--2023-12-26 23:57:21--  https://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-PD.tar
Resolving biomedic.doc.ic.ac.uk (biomedic.doc.ic.ac.uk)... 146.169.22.31
Connecting to biomedic.doc.ic.ac.uk (biomedic.doc.ic.ac.uk)|146.169.22.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4068966400 (3.8G) [application/x-tar]
Saving to: ‘IXI-PD.tar’

IXI-PD.tar          100%[===================>]   3.79G  13.9MB/s    in 5m 39s  

2023-12-27 00:03:01 (11.4 MB/s) - ‘IXI-PD.tar’ saved [4068966400/4068966400]



In [ ]:
! mkdir IXI-PD; tar -xf IXI-PD.tar -C IXI-PD

mkdir: cannot create directory ‘IXI-PD’: File exists


In [ ]:
! python ano-synth/predict.py --flat_latent --name {model_name} --model {model} --root {rootdir} --workers {cores} --amp --data ./IXI-PD --data_name IXI-PD

2023-12-27 00:03:18.725373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 00:03:18.725428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 00:03:18.726650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 00:03:19.759523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Using device: cuda
Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB

Total Images: 578
100% 578/578 [18:47<00:00,  1.95s/it]


## Pathology - Stroke

## Pathology - MS

## Pathology - Glioblastoma

### Data setup

In [14]:
! wget https://msd-for-monai.s3-us-west-2.amazonaws.com/Task01_BrainTumour.tar

--2024-02-01 12:45:01--  https://msd-for-monai.s3-us-west-2.amazonaws.com/Task01_BrainTumour.tar
Resolving msd-for-monai.s3-us-west-2.amazonaws.com (msd-for-monai.s3-us-west-2.amazonaws.com)... 3.5.80.151, 3.5.84.200, 52.218.216.97, ...
Connecting to msd-for-monai.s3-us-west-2.amazonaws.com (msd-for-monai.s3-us-west-2.amazonaws.com)|3.5.80.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7608266240 (7.1G) [application/x-tar]
Saving to: ‘Task01_BrainTumour.tar’

Task01_BrainTumour. 100%[===================>]   7.08G  21.6MB/s    in 5m 37s  

2024-02-01 12:50:38 (21.5 MB/s) - ‘Task01_BrainTumour.tar’ saved [7608266240/7608266240]



In [15]:
! mkdir BRATS; tar -xf Task01_BrainTumour.tar -C BRATS

In [16]:
! mkdir BRATS-T1; mkdir BRATS-T1CE; mkdir BRATS-T2; mkdir BRATS-FLAIR; mkdir BRATS-LABEL

In [17]:
all_data = glob.glob('BRATS/Task01_BrainTumour/imagesTr/*.nii.gz')

In [ ]:
for item in tqdm.tqdm(all_data, total=len(all_data)):
  imgs = nb.load(item)
  data = imgs.get_fdata()
  for i,folder in enumerate(['BRATS-FLAIR', 'BRATS-T1', 'BRATS-T1CE', 'BRATS-T2']):
    img = nb.Nifti1Image(data[...,i], affine=imgs.affine, header=imgs.header)
    nb.save(img, os.path.join(folder, item.split('/')[-1]))
  label = nb.load(item.replace('imagesTr', 'labelsTr'))
  nb.save(label, os.path.join('BRATS-LABEL', item.split('/')[-1]))

  3%|▎         | 15/484 [00:27<13:56,  1.78s/it]

### T1

In [ ]:
! python ano-synth/predict.py --flat_latent --name {model_name} --model {model} --root {rootdir} --workers {cores} --amp --data ./BRATS-T1 --label ./BRATS-LABEL --data_name BRATS-T1

2023-12-27 00:42:25.332698: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 00:42:25.332748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 00:42:25.333946: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 00:42:26.418439: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Using device: cuda
Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB

Total Images: 484
100% 484/484 [19:10<00:00,  2.38s/it]


### T1 (+ Contrast-enhancer)

In [ ]:
! python ano-synth/predict.py --flat_latent --name {model_name} --model {model} --root {rootdir} --workers {cores} --amp --data ./BRATS-T1CE --label ./BRATS-LABEL --data_name BRATS-T1CE

2023-12-27 01:01:46.338066: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 01:01:46.338117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 01:01:46.339309: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 01:01:47.382967: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Using device: cuda
Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB

Total Images: 484
100% 484/484 [19:24<00:00,  2.41s/it]


### T2

In [ ]:
! python ano-synth/predict.py --flat_latent --name {model_name} --model {model} --root {rootdir} --workers {cores} --amp --data ./BRATS-T2 --label ./BRATS-LABEL --data_name BRATS-T2

2023-12-27 01:21:22.003228: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 01:21:22.003274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 01:21:22.004549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 01:21:23.013836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Using device: cuda
Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB

Total Images: 484
100% 484/484 [19:19<00:00,  2.40s/it]


### FLAIR

In [ ]:
! python ano-synth/predict.py --flat_latent --name {model_name} --model {model} --root {rootdir} --workers {cores} --amp --data ./BRATS-FLAIR --label ./BRATS-LABEL --data_name BRATS-FLAIR

2023-12-27 01:40:52.553281: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 01:40:52.553332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 01:40:52.554541: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 01:40:53.577729: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Using device: cuda
Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB

Total Images: 484
100% 484/484 [19:33<00:00,  2.43s/it]
